Лин регрессия хороша тем, можно применить метод градиентного спуска из-за дифференцируемости + выполнена теорема Гаусса-Маркова. С деревьями ничего такого сделать не получится и утверждать тоже ничего нельзя по поводу ее оценки. Имея это ввиду, получается, что лин регрессия лучше дерева решений, когда нужно аппроксимировать функцию, а не просто небольшой набор признаков. То есть лин регрессия будет нормально вести себя на области определения всей функции, а не только на тех участках, где есть тестовая выборка. Кроме того, дерево начинает себя очень плохо вести, когда появляется слишком много признаков - оно разрастается вглубь так, что поиск по нему теперь задача непростая. Лин регрессия же, в свою очередь, будет просто дифференцироваться и идти к точке минимума, это быстрая задача.

In [1]:
import pandas as pd
import numpy as np
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score


In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape

(48192, 3)

In [3]:
X_train = df_train['review']
y_train = df_train['target']

tfidf = TfidfVectorizer(max_df=0.5, min_df=10)
X_train = tfidf.fit_transform(X_train)

X_test = tfidf.transform(df_test['review'])
y_test = df_test['target']

In [4]:
from sklearn.neighbors import KNeighborsClassifier
KNC = KNeighborsClassifier(n_neighbors=2)
KNC.fit(X_train, y_train)
y_pred = KNC.predict(X_test)

In [ ]:
def model_pipeline(model, X_train, X_test, y_train, y_test, hyper_parametrs):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        model.fit(X_train, y_train)

        valid = GridSearchCV(model, param_grid = hyper_parametrs, cv = 5)
        valid.fit(X_train, y_train)

        model = valid.best_estimator_
        print (f"f1_score on train : {model.score(X_train, y_train)} \nAfter regularization: {valid.best_score_} \nBest params: {valid.best_params_} \nTest: {model.score(X_test, y_test)}")

In [ ]:
model_pipeline(KNeighborsClassifier(), X_train, y_train, X_test, y_test, paramsKNN) #недокрутилось

f1_score on train:  0.4517960209485149


In [ ]:
paramsKNN = {'n_neighbors': range(1, 10), 'weights' : ['uniform', 'distance'], 'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']}

In [31]:
KNC = KNeighborsClassifier(weights = 'distance') #лучшее что нашлось вручную
KNC.fit(X_train, y_train)
y_pred = KNC.predict(X_test)
f1_score(y_test, y_pred, average = 'macro')

0.2314787619481402

In [38]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [ ]:
clf = OneVsRestClassifier(SVC()).fit(X_train, y_train) #недокрутилось
y_pred = clf.predict (X_test)
f1_score(y_test, y_pred, average = 'macro')

In [ ]:
ovo = OneVsOneClassifier(SVC()).fit(X, y)
y_pred = clf.predict (X_test)
f1_score(y_test, y_pred, average = 'macro')

In [ ]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=100, n_features=4,n_informative=2, n_redundant=0,random_state=0, shuffle=False)
clf = OutputCodeClassifier(estimator=RandomForestClassifier(random_state=0),random_state=0).fit(X, y)
clf.predict(X_test)
f1_score(y_test, y_pred, average = 'macro')